In [21]:
import requests, yaml
import json
from pprint import pprint
from datetime import datetime, timedelta
import pandas as pd

DISCOURSE_BASE_URL = "https://discourse.onlinedegree.iitm.ac.in"


with open("../key.yaml", "r") as file:
    api_keys = yaml.safe_load(file) 

API_KEY_GLOBAL= api_keys["API_KEY"]
API_USERNAME = 'shubhamG'

QUERY_ID = "103"  # Query id  98
GROUP_NAME = "discourse_analytics"

def get_user_actions(start_date = "22-01-2024", end_date="28-04-2024", category_id=33):
    # API endpoint for Data Explorer
    
    url = f"{DISCOURSE_BASE_URL}/g/{GROUP_NAME}/reports/{QUERY_ID}/run"
    # print(url)

    # Headers required for authentication
    headers = {
        "Accept": "*/*",
        'Api-Key':API_KEY_GLOBAL,
        'Api-Username': API_USERNAME,
        'Content-Type': 'multipart/form-data'
    }

    payload = f'params={{"category_id": {str(category_id)},\
                        "start_date":"22-01-2024",\
                        "end_date":"28-04-2024"}}'

    try:
        response = requests.request("POST", url, headers=headers,data=payload)
        # response = requests.post(url, headers=headers, data=payload)
        # print(response.text)
        response.raise_for_status()  # Raise exception for bad status codes
        data = response.json()

        # Data Explorer returns results in the 'rows' key
        if data.get('rows') and len(data['rows']) > 0:
            return data['rows']  # Returns all rows
        return None

    except Exception as e:
        print(f"Error making request: {e}")
        return None
    
def get_date_ranges(start_date, end_date): # Function to generate date ranges (of 10-day-span) for the given date range
    start_date = datetime.strptime(start_date, "%d-%m-%Y")
    end_date = datetime.strptime(end_date, "%d-%m-%Y")
    date_ranges = []
    while start_date <= end_date:
        end_date_range = start_date + timedelta(days=9)
        if end_date_range > end_date:
            end_date_range = end_date
        date_ranges.append((start_date.strftime("%d-%m-%Y"), end_date_range.strftime("%d-%m-%Y")))
        start_date = end_date_range + timedelta(days=1)
    return date_ranges

In [13]:
columns= ["created_at","user_id","action_type","target_user_id","target_topic_id","target_post_id","topic_title","acting_username","action_name"]
start_date, end_date="22-01-2024", "28-04-2024"
# get the date ranges
date_ranges = get_date_ranges(start_date, end_date)

In [14]:
data=[]
for date_range in date_ranges[:1]:
    print(date_range)
    start_date, end_date = date_range[0], date_range[1]
    rows = get_user_actions(start_date, end_date, category_id=33) # Call the API function
    print(rows[0])
    # data.extend(rows)
    
# df = pd.DataFrame(data, columns=columns)

('22-01-2024', '31-01-2024')
Error making request: 500 Server Error: Internal Server Error for url: https://discourse.onlinedegree.iitm.ac.in/g/discourse_analytics/reports/103/run


TypeError: 'NoneType' object is not subscriptable

In [22]:
# get_user_actions('22-01-2024', '31-01-2024')
get_user_actions()

Error making request: 500 Server Error: Internal Server Error for url: https://discourse.onlinedegree.iitm.ac.in/g/discourse_analytics/reports/103/run
